<a href="https://colab.research.google.com/github/Pinkysol/Learning/blob/main/FAST_API_MNIST_REQUEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В домашней работе необходимо с помощью **FastAPI** реализовать **REST API**:

1. На 3 балла. Твой **REST API** - это список покупок и содержит поля: название товара, группа товара (например, электроника или продовольствие), цена, единица измерения, количество. Также необходимо реализовать метод, который возвращает список - расходы по каждой группе товаров и сумму всех покупок. Также необходимо с помощью библиотеки *requests* продемонстрировать запросами к **REST API**, как работает ваш веб-сервис. Это задание можно сравнить с "покрытием тестами" вашего API. Нечто похожее делают тестировщики в ИТ-компаниях. Вам необходимо покрыть запросами все методы, которые вы реализуете на веб-сервере.
2. **На 4 балла**. Вам необходимо сделать красивую документацию для твоего **REST API** с подробным описанием. Для этого тебе придется обратиться к документации:
  
  https://fastapi.tiangolo.com/ru/tutorial/metadata/
  
  https://fastapi.tiangolo.com/ru/tutorial/path-operation-configuration/#response-description

3. **На 5 баллов**. Творческое задание. **REST API** можно использовать для взаимодействия с вашей моделью нейронной сети. Вы уже знаете, что можно обучить модель, а лучший результат выгрузить для дальнейшего использования. Для получения 5 баллов необходимо обучить свою модель, загрузить ее в **Colab**. Задача может быть любой: регрессии, классификации, входными данными могут быть картинки или текстовые данные. С помощью **REST API** обеспечьте взаимодействие с моделью. Это полностью творческое задание!

In [21]:
# Импортируем фреймворк Keras
import keras
from keras.datasets import mnist
# Библиотека AutoML autokeras
import autokeras as ak
# Утилита для расщепления выборки
from sklearn.model_selection import train_test_split

(x_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(x_train,
  y_train, test_size=0.2, stratify=y_train, random_state=42)

clf = ak.ImageClassifier(overwrite=True, max_trials=1)
result_training = clf.fit(X_train, y_train, epochs = 1, validation_data = (X_val, y_val))
model = clf.export_model()
model.save("model_autokeras.keras")

Trial 1 Complete [00h 00m 08s]
val_loss: 0.05802246183156967

Best val_loss So Far: 0.05802246183156967
Total elapsed time: 00h 00m 08s
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8887 - loss: 0.3536 - val_accuracy: 0.9815 - val_loss: 0.0654


/home/dimas/miniconda3/envs/tensor/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [45]:
%%writefile main.py
import io
import pickle
import numpy as np
import PIL.Image
import PIL.ImageOps
from keras.models import load_model
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import autokeras as ak

app = FastAPI()

model = load_model(
    "model_autokeras.keras", custom_objects=ak.CUSTOM_OBJECTS
)

@app.post("/number_predict")
async def predict_image(file: UploadFile = File(...)):
    contents = await file.read()

    pil_image = PIL.Image.open(io.BytesIO(contents))

    pil_image = pil_image.resize((28, 28), PIL.Image.Resampling.LANCZOS)

    img_array = np.array(pil_image)
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    predicted_value = prediction[0].argmax(axis=0)
    return {"prediction": str(predicted_value)}

Overwriting main.py


In [53]:
import requests
import io
from keras.models import load_model

with open("img_2.jpg", 'rb') as f:
    img_data = f.read()

files = {'file': img_data}
r = requests.post('http://127.0.0.1:8000/number_predict', files=files)

print(r)
number = r.json()
print(number)

<Response [200]>
{'prediction': '0'}
